## 白血病データに対し、ニュートンラフソン法による最尤法を実施

In [34]:
import numpy as np
import pandas as pd

d = ｐｄ.read_csv("/Users/ShuntaroMiwa/leukemia.csv", encoding="SHIFT_JIS")
beta0 = 62.5
beta1 = 0
n = len(d) 
xmean= ｄ["x"].mean()
xd = np.array([d["x"] - xmean])
xd = xd.reshape(17, 1)
y = np.array(d["T"])
y = y.reshape(17, 1)
#print(xd)

In [35]:
#このページを参考に作成（https://python.quantecon.org/mle.html）

%matplotlib inline
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (11, 5)  #set default figure size
import numpy as np
from numpy import exp
from scipy.special import factorial
import pandas as pd
from mpl_toolkits.mplot3d import Axes3D
import statsmodels.api as sm
from statsmodels.api import Poisson
from scipy import stats
from scipy.stats import norm
from statsmodels.iolib.summary2 import summary_col
import sympy as sym
from sympy import *

class ExpoRegression:
    
    def __init__(self, y, xd, β):
        self.xd = xd
        self.n = 17
        self.y = y
        self.β = β.reshape(2,1)

    def μ(self):
        return (((self.β[0]*np.exp(self.β[1]*self.xd))**-1)*np.exp(-1*self.β[0]*np.exp(self.β[1]*self.xd)**-1))

    def logL(self):
        y = self.y
        μ = self.μ()
        return - 1*np.log(float(self.β[0]))*self.n +np.sum(-1* self.β[1]*xd- y*(1/self.β[0])*np.exp(-1*self.β[1]*xd))

    def G(self):
        f = - 1*np.log(self.β[0])*self.n +np.sum(-1* self.β[1]*xd- y*(1/self.β[0])*np.exp(-1*self.β[1]*xd))
        f_β0 = -1*self.n*(self.β[0]**-1)+np.sum(y*(self.β[0]**-2)*np.exp(-1*self.β[1]*xd))
        f_β1 = np.sum(-xd+xd*y*(self.β[0]**-1)*np.exp(-1*self.β[1]*xd))
        return np.array([[*f_β0], [f_β1]])

    def H(self):
        f_β0β0 = self.n*(self.β[0]**-2)+np.sum(-2*y*(self.β[0]**-3)*np.exp(-self.β[1]*xd))
        f_β0β1 = np.sum(-1*xd*y*(self.β[0]**-2)*np.exp(-1*self.β[1]*xd))
        f_β1β0 = np.sum(-1*xd*y*(self.β[0]**-2)*np.exp(-1*self.β[1]*xd))
        f_β1β1 = np.sum(-1*(xd**-2)*y*(self.β[0]**-1)*np.exp(-1*self.β[1]*xd))
        return np.array([[*f_β0β0,f_β0β1],[f_β1β0 ,f_β1β1]])

In [42]:
def newton_raphson(model, tol=-1, max_iter=1000, display=True):

    i = 0
    error = 100  # Initial error value

    # Print header of output
    if display:
        header = f'{"Iteration_n":<13}{"Log-likelihood":<16}{"β0,β":<60}'
        print(header)
        print("-" * len(header))

    # While loop runs while any value in error is greater
    # than the tolerance until max iterations are reached
    while np.any(error > tol) and i < max_iter:
        H, G = model.H(), model.G()
        β_new = model.β - (np.linalg.inv(H) @ G)
        error = β_new - model.β
        model.β = β_new

        # Print iterations
        if display:
            β_list = [f'{t:.3}' for t in list(model.β.flatten())]#flatten：一次元配列にする
            update = f'{i:<13}{model.logL():<16.8}{β_list}'
            print(update)

        i += 1

    print(f'Number of iterations: {i}')
    print(f'β_hat = {model.β.flatten()}')

    # Return a flat array for β (instead of a k_by_1 column vector)
    return model.β.flatten()

In [43]:
init_β = np.array([62.5, 0])

# f_β0 = -1*n*(Beta[0]**-1)+np.sum(y*(Beta[0]**-2)*np.exp(-1*Beta[1]*xd))
# f_β1 = np.sum(-xd+xd*y*(Beta[0]**-1)*np.exp(-1*Beta[1]*xd))

# G= np.array([[*f_β0], [f_β1]])

# f_β0β0 = n*(Beta[0]**-2)+np.sum(-2*y*(Beta[0]**-3)*np.exp(-Beta[1]*xd))
# f_β0β1 = np.sum(-1*xd*y*(Beta[0]**-2)*np.exp(-1*Beta[1]*xd))
# f_β1β0 = np.sum(-1*xd*y*(Beta[0]**-2)*np.exp(-1*Beta[1]*xd))
# f_β1β1 = np.sum(-1*(xd**-2)*y*(Beta[0]**-1)*np.exp(-1*Beta[1]*xd))

# H =np.array([[*f_β0β0,f_β0β1],
#          [f_β1β0 ,f_β1β1]])

# Create an object with Poisson model values
expo = ExpoRegression(y, xd, β=init_β)

# Use newton_raphson to find the MLE
β_hat = newton_raphson(expo, display=True)

Iteration_n  Log-likelihood  β0,β1                                                       
-----------------------------------------------------------------------------------------
0            -87.234034      ['62.3', '-0.00939']
1            -87.179245      ['62.1', '-0.0187']
2            -87.125413      ['61.9', '-0.0279']
3            -87.07252       ['61.7', '-0.037']
4            -87.020546      ['61.5', '-0.046']
5            -86.969472      ['61.3', '-0.055']
6            -86.919281      ['61.1', '-0.0638']
7            -86.869954      ['60.9', '-0.0726']
8            -86.821475      ['60.8', '-0.0812']
9            -86.773826      ['60.6', '-0.0898']
10           -86.726992      ['60.4', '-0.0983']
11           -86.680956      ['60.3', '-0.107']
12           -86.635702      ['60.1', '-0.115']
13           -86.591217      ['60.0', '-0.123']
14           -86.547484      ['59.8', '-0.132']
15           -86.504489      ['59.6', '-0.14']
16           -86.462218      ['59.5', '-0.14